Метрики на FJSP

In [1]:
from src.datasets import FJSPBenchmarksDataset

from src.solvers.jsp.fjsp.heuristic.dispatching_rules import FJSPFIFOSolver, FJSPMOPNRSolver, FJSPSTPSolver, FJSPMWKRSolver
from src.solvers.jsp.fjsp.heuristic.random_search import FJSPRandomSolver
from src.solvers.jsp.fjsp.exact.or_tools import FJSPORToolsSolver
from src.solvers.jsp.fjsp.rl4co.l2d import FJSPL2DSolver
from tqdm import tqdm

from time import time

import pandas as pd

import torch

/home/daniil/programming/diplom/mtrlgnn/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
bench_list = ["Brandimarte", "Hurink_edata", "Hurink_rdata", "Hurink_vdata"]

In [3]:
results = {}
times_results = {}

In [4]:
for bench in bench_list:
    dataset = FJSPBenchmarksDataset(file_pattern=f"../data/jsp/benchmarks/{bench}/*.fjs")
    print(bench)
    costs = {"FJSPFIFOSolver": [], "FJSPMOPNRSolver": [], "FJSPSTPSolver": [], "FJSPMWKRSolver": [], "Best DR": []}
    times = {"FJSPFIFOSolver": [], "FJSPMOPNRSolver": [], "FJSPSTPSolver": [], "FJSPMWKRSolver": [], "Best DR": []}
    for inst, _ in tqdm(dataset):
        res = []
        full_time = 0

        for solver_name, solver in [("FJSPFIFOSolver", FJSPFIFOSolver()), ("FJSPMOPNRSolver", FJSPMOPNRSolver()), 
                                    ("FJSPSTPSolver", FJSPSTPSolver()), ("FJSPMWKRSolver", FJSPMWKRSolver())]:
            
            try:

                start = time()
                solution = solver.solve(inst)
                cost = inst.total_cost(solution)
                times[solver_name].append(time()-start)
                costs[solver_name].append(cost)
                
                full_time += time()-start
                res.append(cost)

            except AssertionError:
                print(solver_name)


        costs["Best DR"].append(min(res))
        times["Best DR"].append(full_time)

    results[bench] = {solver: sum(stats)/len(stats) for solver, stats in costs.items()}
    times_results[bench] = {solver: sum(stats)/len(stats) for solver, stats in times.items()}

Brandimarte


100%|██████████| 10/10 [00:20<00:00,  2.06s/it]


Hurink_edata


100%|██████████| 40/40 [01:31<00:00,  2.28s/it]


Hurink_rdata


100%|██████████| 40/40 [01:29<00:00,  2.23s/it]


Hurink_vdata


100%|██████████| 40/40 [01:31<00:00,  2.30s/it]


In [5]:
results_df  = pd.DataFrame(results).T
results_df

,FJSPFIFOSolver,FJSPMOPNRSolver,FJSPSTPSolver,FJSPMWKRSolver,Best DR
Brandimarte,250.400,205.800,239.600,200.800,197.40
Hurink_edata,1387.075,1206.425,1304.325,1203.150,1174.95
Hurink_rdata,1211.125,1061.075,1193.725,1045.425,1030.15
Hurink_vdata,1078.550,969.550,1079.275,947.650,943.90


In [6]:
times_results_df = pd.DataFrame(times_results).T
times_results_df

,FJSPFIFOSolver,FJSPMOPNRSolver,FJSPSTPSolver,FJSPMWKRSolver,Best DR
Brandimarte,0.510726,0.512156,0.496552,0.531978,2.051420
Hurink_edata,0.560045,0.563278,0.548919,0.599114,2.271363
Hurink_rdata,0.558153,0.550540,0.532378,0.573923,2.215000
Hurink_vdata,0.562929,0.565385,0.553577,0.602010,2.283907


In [7]:
df_full_result = pd.concat([results_df.T, times_results_df.T], axis=1, keys=['Cost', 'Time'])

In [8]:
df_full_result.swaplevel(axis=1).sort_index(axis=1)

Brandimarte           Hurink_edata           Hurink_rdata  \
                       Cost      Time         Cost      Time         Cost   
FJSPFIFOSolver        250.4  0.510726     1387.075  0.560045     1211.125   
FJSPMOPNRSolver       205.8  0.512156     1206.425  0.563278     1061.075   
FJSPSTPSolver         239.6  0.496552     1304.325  0.548919     1193.725   
FJSPMWKRSolver        200.8  0.531978     1203.150  0.599114     1045.425   
Best DR               197.4  2.051420     1174.950  2.271363     1030.150   

                          Hurink_vdata            
                     Time         Cost      Time  
FJSPFIFOSolver   0.558153     1078.550  0.562929  
FJSPMOPNRSolver  0.550540      969.550  0.565385  
FJSPSTPSolver    0.532378     1079.275  0.553577  
FJSPMWKRSolver   0.573923      947.650  0.602010  
Best DR          2.215000      943.900  2.283907

In [4]:
results = {}
times_results = {}

In [9]:
for bench in bench_list:
    dataset = FJSPBenchmarksDataset(file_pattern=f"../data/jsp/benchmarks/{bench}/*.fjs")
    print(bench)
    costs = {"FJSPORToolsSolver": []}
    times = {"FJSPORToolsSolver": []}
    for inst, _ in tqdm(dataset):

        for solver_name, solver in [("FJSPORToolsSolver", FJSPORToolsSolver(time_limit=10))]:
            # try:

            start = time()
            solution = solver.solve(inst)
            cost = inst.total_cost(solution)
            costs[solver_name].append(cost)
            times[solver_name].append(time()-start)

            # except AssertionError as e:
            #     print(solver_name, e)

    results[bench] = {solver: sum(stats)/len(stats) for solver, stats in costs.items()}
    times_results[bench] = {solver: sum(stats)/len(stats) for solver, stats in times.items()}

Brandimarte


100%|██████████| 10/10 [01:11<00:00,  7.18s/it]


Hurink_edata


100%|██████████| 40/40 [03:27<00:00,  5.20s/it]


Hurink_rdata


100%|██████████| 40/40 [06:20<00:00,  9.50s/it]


Hurink_vdata


100%|██████████| 40/40 [06:40<00:00, 10.00s/it]


In [10]:
results_df  = pd.DataFrame(results).T
results_df

,FJSPORToolsSolver
Brandimarte,182.800
Hurink_edata,1039.125
Hurink_rdata,966.325
Hurink_vdata,1063.500


In [12]:
times_results_df = pd.DataFrame(times_results).T
times_results_df

,FJSPORToolsSolver
Brandimarte,7.170302
Hurink_edata,5.194067
Hurink_rdata,9.499462
Hurink_vdata,9.993426


In [13]:
df_full_result = pd.concat([results_df.T, times_results_df.T], axis=1, keys=['Cost', 'Time'])

In [14]:
df_full_result.swaplevel(axis=1).sort_index(axis=1)

Brandimarte           Hurink_edata           Hurink_rdata  \
                         Cost      Time         Cost      Time         Cost   
FJSPORToolsSolver       182.8  7.170302     1039.125  5.194067      966.325   

                            Hurink_vdata            
                       Time         Cost      Time  
FJSPORToolsSolver  9.499462       1063.5  9.993426

In [3]:
results = {}
times_results = {}

In [4]:
accelerator = "cpu"
batch_size = 32
train_data_size = 1_000
embed_dim = 64
num_encoder_layers = 2

In [5]:
l2d_solver = FJSPL2DSolver(train_data_size=train_data_size, batch_size=batch_size, max_epochs=15, accelerator=accelerator, lr=0.0001, val_data_size=1_000, 
                       embed_dim=embed_dim, num_encoder_layers=num_encoder_layers)

In [6]:
checkpoint = torch.load("./checkpoints/l2d.pt", weights_only=False)

In [7]:
l2d_solver.policy = checkpoint

In [8]:
for bench in bench_list:
    dataset = FJSPBenchmarksDataset(file_pattern=f"../data/jsp/benchmarks/{bench}/*.fjs")
    print(bench)
    costs = {"FJSPL2DSolver": []}
    times = {"FJSPL2DSolver": []}
    for inst, _ in tqdm(dataset):

        for solver_name, solver in [("FJSPL2DSolver", l2d_solver)]:
            
            try:

                start = time()
                solution = solver.solve(inst)
                cost = inst.total_cost(solution)
                costs[solver_name].append(cost)
                times[solver_name].append(time()-start)

            except AssertionError as e:
                print(solver_name, e)


    results[bench] = {solver: sum(stats)/len(stats) for solver, stats in costs.items()}
    times_results[bench] = {solver: sum(stats)/len(stats) for solver, stats in times.items()}

Brandimarte


100%|██████████| 10/10 [00:06<00:00,  1.67it/s]


Hurink_edata


100%|██████████| 40/40 [00:25<00:00,  1.58it/s]


Hurink_rdata


100%|██████████| 40/40 [00:25<00:00,  1.56it/s]


Hurink_vdata


100%|██████████| 40/40 [00:25<00:00,  1.56it/s]


In [9]:
results_df  = pd.DataFrame(results).T
results_df

,FJSPL2DSolver
Brandimarte,209.40
Hurink_edata,1291.55
Hurink_rdata,1075.40
Hurink_vdata,991.70


In [10]:
times_results_df = pd.DataFrame(times_results).T
times_results_df

,FJSPL2DSolver
Brandimarte,0.595196
Hurink_edata,0.630493
Hurink_rdata,0.635453
Hurink_vdata,0.632810


In [11]:
df_full_result = pd.concat([results_df.T, times_results_df.T], axis=1, keys=['Cost', 'Time'])

In [12]:
df_full_result.swaplevel(axis=1).sort_index(axis=1)

Brandimarte           Hurink_edata           Hurink_rdata  \
                     Cost      Time         Cost      Time         Cost   
FJSPL2DSolver       209.4  0.595196      1291.55  0.630493       1075.4   

                        Hurink_vdata           
                   Time         Cost     Time  
FJSPL2DSolver  0.635453        991.7  0.63281

In [4]:
results = {}
times_results = {}

In [6]:
for bench in bench_list:
    dataset = FJSPBenchmarksDataset(file_pattern=f"../data/jsp/benchmarks/{bench}/*.fjs")
    print(bench)
    costs = {"FJSPRandomSolver": []}
    times = {"FJSPRandomSolver": []}
    for inst, _ in tqdm(dataset):

        for solver_name, solver in [("FJSPRandomSolver", FJSPRandomSolver(n_search=100))]:
            
            try:

                start = time()
                solution = solver.solve(inst)
                cost = inst.total_cost(solution)
                times[solver_name].append(time()-start)
                costs[solver_name].append(cost)

            except AssertionError:
                print(solver_name)

    results[bench] = {solver: sum(stats)/len(stats) for solver, stats in costs.items()}
    times_results[bench] = {solver: sum(stats)/len(stats) for solver, stats in times.items()}

Brandimarte


100%|██████████| 10/10 [04:07<00:00, 24.79s/it]


Hurink_edata


100%|██████████| 40/40 [18:05<00:00, 27.13s/it]


Hurink_rdata


100%|██████████| 40/40 [18:12<00:00, 27.32s/it]


Hurink_vdata


100%|██████████| 40/40 [17:59<00:00, 27.00s/it]


In [7]:
results_df  = pd.DataFrame(results).T
results_df

,FJSPRandomSolver
Brandimarte,203.900
Hurink_edata,1165.825
Hurink_rdata,1032.325
Hurink_vdata,951.675


In [8]:
times_results_df = pd.DataFrame(times_results).T
times_results_df

,FJSPRandomSolver
Brandimarte,24.785972
Hurink_edata,27.121553
Hurink_rdata,27.310810
Hurink_vdata,26.992061


In [9]:
df_full_result = pd.concat([results_df.T, times_results_df.T], axis=1, keys=['Cost', 'Time'])

In [10]:
df_full_result.swaplevel(axis=1).sort_index(axis=1)

Brandimarte            Hurink_edata            Hurink_rdata  \
                        Cost       Time         Cost       Time         Cost   
FJSPRandomSolver       203.9  24.785972     1165.825  27.121553     1032.325   

                           Hurink_vdata             
                      Time         Cost       Time  
FJSPRandomSolver  27.31081      951.675  26.992061